In [1]:
using Pkg
Pkg.activate("/home/wkh/XLA.jl")

"/home/wkh/XLA.jl/Project.toml"

In [2]:
using Distributions
Pkg.resolve()
using Distributions

┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1186
┌ Warning: Package Distributions does not have Test in its dependencies:
│ - If you have Distributions checked out for development and have
│   added Test as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Distributions
└ Loading Test into Distributions from project dependency, future warnings for Distributions are suppressed.


  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/XLA.jl/Project.toml`
 [no changes]
  Updating `~/XLA.jl/Manifest.toml`
 [no changes]


In [4]:
using XLA, TensorFlow, Plots, BenchmarkTools

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1186


In [5]:
sess = Session(Graph(); target="grpc://10.240.1.2:8470")
run(sess, TensorFlow.Ops.configure_distributed_tpu())
XLA.initialize!("10.240.1.2:8470"; reset=true);

2019-11-15 18:12:21.790443: W tensorflow/core/distributed_runtime/rpc/grpc_session.cc:349] GrpcSession::ListDevices will initialize the session with an empty graph and other defaults because the session has not yet been created.


In [6]:
f(A) = A*A*A

f (generic function with 1 method)

In [7]:
hostA = randn(Float32, 10000, 10000)
A = tpu(hostA);

In [8]:
Pkg.add("BenchmarkTools")
using BenchmarkTools

 Resolving package versions...
 Installed BenchmarkTools ─ v0.4.3
  Updating `~/XLA.jl/Project.toml`
  [6e4b80f9] + BenchmarkTools v0.4.3
  Updating `~/XLA.jl/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v0.4.3


┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1186


In [10]:
@time f(hostA);

  8.765164 seconds (8 allocations: 762.940 MiB, 0.50% gc time)


In [11]:
@time f(hostA);

  9.591934 seconds (8 allocations: 762.940 MiB, 2.05% gc time)


In [13]:
@time @tpu f(A)

TensorFlow.TFException: Tensorflow error: Status: Resource tuples/8917215225179496512/N10tensorflow18XRTTupleAllocationE does not exist.
	 [[{{node XRTExecute_2}} = XRTExecute[Ninputs=1, _device="/job:tpu_worker/replica:0/task:0/device:TPU:0"](XRTExecute_2/Const_5_S5, Const_4_G29, XRTExecute_2/Const_6_S7)]]
	 [[{{node XRTExecute_2_G31}} = _Recv[client_terminated=false, recv_device="/job:tpu_worker/replica:0/task:0/device:CPU:0", send_device="/job:tpu_worker/replica:0/task:0/device:TPU:0", send_device_incarnation=2137547159556300961, tensor_name="edge_8_XRTExecute_2", tensor_type=DT_INT64, _device="/job:tpu_worker/replica:0/task:0/device:CPU:0"]()]]


In [ ]:
g() = randn(XRTArray{Float32}, 10000, 10000)
@benchmark @tpu g()

In [ ]:
h() = f(g())
@benchmark @tpu h()

In [ ]:
relu(x) = max(x, zero(x))

In [ ]:
plot(relu, -1.0, 1.0)

In [ ]:
r() = relu.(randn(XRTArray{Float32}, 16, 16))
@tpu r()

In [ ]:
struct Dense{T,S,F}
    W::T
    b::S
    σ::F
end

Base.show(io::IO, d::Dense) = print(io, "Dense", size(d.W))
(d::Dense)(x) = d.σ.(d.W*x + d.b)

In [ ]:
make_one(A) = Dense(A, zeros(XRTArray{Float32, (16,), 1}), relu)

In [ ]:
dense = @tpu make_one(@tpu r())

In [ ]:
df(dense) = dense(randn(XRTArray{Float32}, 16))

In [ ]:
@tpu df(dense)